In [1]:
import pandas as pd
import json
from datasets import DatasetDict, Dataset, load_dataset, load_from_disk
from transformers import AutoTokenizer
from dataprocessor import SamplePreprocessorForFinetune

/home/qiaoan/miniconda3/envs/LIU/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_dataset = load_dataset("json", data_files="/home/qiaoan/data/TableInstruct/TableInstruct_instructions.jsonl", split="train")
print(len(train_dataset))
print(train_dataset[0])

19661
{'id': '842e15a0ea54a3bb7f7cf13c8cba28dc', 'qtype': 'FactChecking', 'qsubtype': 'MatchBased', 'instruction': 'You are a data analyst proficient in Python. Your task is to write executable Python code to analyze the table and then answer questions.\n\n[Guidelines]\nYou should act following requirements below:\n1. based on the question, write out your analytical approach, and then write Python code according to this approach.\n2. The code needs to be concise and easy to understand, and if necessary, add comments for clarification.\n3. Code blocks need to strictly start with ```python and end with ```\n4. Your analysis must be based entirely on the above data. If the user\'s question is not related to data analysis, please politely refuse.\n5. You need to generate executable code. If there are results to be presented, please use the print function; if there are charts, please use the matplotlib library to draw them.\n6. Ensure to load the table with command ```df = pd.read_csv(\'tab

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/home/qiaoan/data/Qwen2-7B-Instruct")
preprocessor = SamplePreprocessorForFinetune(tokenizer, max_length=3000)
samples = []
from tqdm import tqdm
for sample in tqdm(train_dataset):
    try:
        table_dict = json.loads(sample['table'])
        df = pd.DataFrame(table_dict["data"], columns=table_dict["columns"])
    except:
        continue
    sample['df'] = df
    processed_sample = preprocessor(sample)
    samples.append(processed_sample)
train_dataset = Dataset.from_list(samples)
# train_dataset.save_to_disk(data_args.save_path)
train_dataset.shuffle(seed=42)
eval_dataset = None
# filter out training sample that token length larger than 10000
train_dataset = train_dataset.filter(lambda x: len(x['input_ids']) <= 6000, num_proc=32)  
print(len(train_dataset))
# for sample in train_dataset:
#     sample = sample_preprocessor(sample)
#     print(len(sample['input_ids']), len(sample['label_ids']))

Filter (num_proc=32): 100%|██████████| 19655/19655 [02:05<00:00, 156.67 examples/s]


19604


In [6]:
df = pd.DataFrame(columns=['A', 'B', 'C'], data=[[1, 2, 3], [4, 5, 6], [7, 8, 9]])
string = df.iloc[:2].to_csv(index=False, header=True)
print(string)

A,B,C
1,2,3
4,5,6

